# DDM: Model Validation

This notebook provides an overview of the tools built in the DDM for extracting predictions from your trained DDM and for evauating the performance of the DDM.

---

We utilize `hydra` to save the configuration of our datasets and our models. The default configuration is in the `conf/config.yaml` directory:

```YAML
defaults:
  - data: cartpole_st1_at.yaml
  - model: SVR.yaml
  - simulator: gboost_cartpole.yaml
```

Note that the configuration file points to three additional configuration files for each component: the data, the model, and the simulator.

While the configuration file already has default values specified you can override any element of the configuration file using the `overrides` option. For example, we override the data configuration to instead use the `yaml` file specified in `data/cartpole-st_at.yaml` and the model configuration to use the `yaml` file specified in `model/xgboost.yaml`.


In [2]:
cd ..

C:\microsoft-datadrivenmodel


In [ ]:
from hydra.experimental import initialize, compose
from model_loader import available_models
import logging
import matplotlib.pyplot as plt
import numpy as np
from rich import print
from rich.logging import RichHandler
import copy

logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler()]
)
logger = logging.getLogger("ddm_validation")
logger.setLevel(logging.INFO)

In [ ]:
initialize(config_path="../conf", job_name="model_validation")
cfg = compose(config_name="config", overrides=["data=cartpole_st_at", "model=xgboost"])

## 1. Importing the Dataset

In [ ]:
cfg["data"]

In [ ]:
cfg["model"]

In [ ]:
# Extract features from yaml file
input_cols = cfg['data']['inputs']
output_cols = cfg['data']['outputs']
augmented_cols = cfg['data']['augmented_cols']
dataset_path = cfg['data']['path']
iteration_order = cfg['data']['iteration_order']
episode_col = cfg['data']['episode_col']
iteration_col = cfg['data']['iteration_col']
max_rows = cfg['data']['max_rows']

In [ ]:
print("DATA STRUCTURE SELECTED:")
print(" - input_cols:", input_cols)
print(" - augmented_cols:", augmented_cols)
print(" - output_cols:", output_cols)

##  2. Model Definition

The `available_models` dictionary provides wrappers for the available models in this repository. We utilize `cfg["model"]` to load and build the model specified in the `model.yaml` file.

### Training Hyperparameters

Every model has its own hyperparameters, specified through the `cfg["model"]["build_params"]` dictionary, which can be modified directly in the dictionary below or through the `hydra` overrides.

In [ ]:
cfg["model"]["build_params"]

## 3. Train the Model

In [ ]:
def train_models(config=cfg):

    logger.info(f'Model type: {available_models[config["model"]["name"]]}')
    Model = available_models[config["model"]["name"]]
    model = Model()
    logger.info(f"Building model with parameters: {config}")
    model.build_model(
        **config["model"]["build_params"]
    )
    logger.info(f"Loading data from {dataset_path}")
    global X, y
    X, y = model.load_csv(
        input_cols=input_cols,
        output_cols=output_cols,
        augm_cols=list(augmented_cols),
        dataset_path=dataset_path,
        iteration_order=iteration_order,
        episode_col=episode_col,
        iteration_col=iteration_col,
        max_rows=max_rows,
    )

    logger.info(f"Fitting model...")
    model.fit(X, y)
    logger.info(f"Model trained!")

    return model

In [ ]:
model = train_models(cfg)

### Save Model

In [ ]:
model.save_model(filename=cfg["model"]["saver"]["filename"])

### Data Structure of Saved model

In [ ]:
logger.info(f"Input_cols:  {model.features}")
logger.info(f"Output_cols: {model.labels}")

## 4. Model Evaluations

We provide three methods for evaluating the errors of our trained models:

1. Model predictive error: using a specified metric (such as RMSE) and a test set, we evaluate the metric on the test set.
2. Visualization of per-iteration predictions vs a test set
3. Visualization of sequential predictions vs a test set

### 4.1. Estimated Prediction Error
#### 4.1.a. Estimated Prediction Error: Per-Iteration Predictions

In [ ]:
from assessment_metrics_loader import available_metrics
rms_error = available_metrics["root_mean_squared_error"]
ms_error = available_metrics["mean_squared_error"]

In [ ]:
X_test, y_test = model.get_test_set(grouped_per_episode=False)

per_iteration_eval_table = model.evaluate(rms_error, X_test, y_test, marginal=True)

per_iteration_eval_table.rename(columns = {'score':'score_per_it'}, inplace = True)

In [ ]:
if (per_iteration_eval_table["score_per_it"] > 1.0).any():
    logger.warn("Per-iteration assessment error is high. Please, review your model")

per_iteration_eval_table

#### 4.1.b. Estimated Prediction Error: Per-Iteration Predictions

In [ ]:
# number of iterations to predict sequentially
it_per_episode = 50

In [ ]:
X_test_grouped, y_test_grouped = model.get_test_set(grouped_per_episode=True)
print(type(X_test_grouped))

sequential_eval_table = model.evaluate_sequentially(
    rms_error,
    X_test_grouped,
    y_test_grouped,
    marginal=True,
    it_per_episode=it_per_episode
)

sequential_eval_table.rename(columns = {'score':'seq_score'}, inplace = True)

In [ ]:
if (sequential_eval_table["seq_score"] > 1.0).any():
    logger.warn("Sequentially analyzed assessment error is high. Please, review your model")

sequential_eval_table

### 4.2. Per-Iteration Predictions

In [ ]:
action_feat = "action_command"

In [ ]:
X_test, y_test = model.get_test_set(grouped_per_episode=False)
preds = model.predict(X_test)

In [ ]:
# Plot results graphs
label_count = np.shape(preds)[1]
fig,axs = plt.subplots(3,2,)
for i in range(label_count):
    axs[i//2,i%2].set_title(f"Per-iteration predictions: {model.labels[i]}")
    axs[i//2,i%2].plot(y_test[:,0], "green")
    axs[i//2,i%2].plot(preds[:,i], "brown")
    
# select section to zoom in (for both action & episode visualization)
sample_from = 0
sample_to = 100

# Plot action changes
action_idx = model.features.index(action_feat)
axs[(i+1)//2,(i+1)%2].set_title(f"Plot [{sample_from}:{sample_to}] actions (ensure action is not stale)")
axs[(i+1)//2,(i+1)%2].plot(X[sample_from:sample_to,action_idx])
    
# Plot prediction graph -- with zoom on 100 iterations
action_idx = model.features.index(action_feat)
feat_idx = 0
axs[(i+2)//2,(i+2)%2].set_title(f"Plot [{sample_from}:{sample_to}] state preds: {model.labels[feat_idx]}")
axs[(i+2)//2,(i+2)%2].plot(y_test[sample_from:sample_to,feat_idx], "green")
axs[(i+2)//2,(i+2)%2].plot(preds[sample_from:sample_to,feat_idx], "brown")

### 4.3. Sequential Predictions

In [ ]:
# number of iterations to predict sequentially
# - taking the value defined above by default -
it_per_episode = it_per_episode
action_feat = action_feat

In [ ]:
X_test_grouped, y_test_grouped = model.get_test_set(grouped_per_episode=True)

preds_sequentially, labels_sequentially = model.predict_sequentially(
    X_test_grouped,
    y_test_grouped,
    it_per_episode=it_per_episode
)

In [ ]:
# Plot results graphs
label_count = np.shape(preds_sequentially)[1]
fig,axs = plt.subplots(3,2,)
for i in range(label_count):
    axs[i//2,i%2].set_title(f"Sequential predictions: {model.labels[i]}")
    axs[i//2,i%2].plot(labels_sequentially[:,0], "green")
    axs[i//2,i%2].plot(preds_sequentially[:,i], "brown")
    
# select section to zoom in (for both action & episode visualization)
sample_from = 0
sample_to = 100

# Plot action changes
action_idx = model.features.index(action_feat)
axs[(i+1)//2,(i+1)%2].set_title(f"Plot [{sample_from}:{sample_to}] actions (ensure action is not stale)")
axs[(i+1)//2,(i+1)%2].plot(X[sample_from:sample_to,action_idx])
    
# Plot prediction graph -- with zoom on 100 iterations
action_idx = model.features.index(action_feat)
feat_idx = 0
axs[(i+2)//2,(i+2)%2].set_title(f"Plot [{sample_from}:{sample_to}] state preds: {model.labels[feat_idx]}")
axs[(i+2)//2,(i+2)%2].plot(labels_sequentially[sample_from:sample_to,feat_idx], "green")
axs[(i+2)//2,(i+2)%2].plot(preds_sequentially[sample_from:sample_to,feat_idx], "brown")

## 5. Comparing Model Evaluations

If you want to compare various models, you can use the following section to save them in between runs.

1. Select appropriate "model_name" tag, and run this section
2. Change config through "config.yaml" (located at 'conf' folder)
3. Rerun from Model Build, until this section
4. Define a new value for "model_name" tag, and run this section again

In [ ]:
# select model name, and feats to extract
model_name = "xgb"

In [ ]:
# per-iteration & sequential scores
model_per_it_scores = copy.deepcopy(per_iteration_eval_table)
model_sequential_scores = copy.deepcopy(sequential_eval_table)

In [ ]:
# initialize models dictionary if it doesn't exist already
if 'models_dict' not in locals():
    models_dict = dict()

# append tables to model using selected model name as key
models_dict[model_name] = (model_per_it_scores, model_sequential_scores)

# redefine column names if needed
for model_name, score_tables in models_dict.items():
    for score_table in score_tables:
        for col_name in score_table.columns:
            if "score" in col_name and model_name not in col_name:
                score_table.rename(columns = {col_name:model_name+"_"+col_name}, inplace = True)

# concatenate across all models
all_scores = None
for model_name, score_tables in models_dict.items():
    for score_table in score_tables:
        if all_scores is None:
            all_scores = score_table
        else:
            all_scores = all_scores.merge(score_table,how='outer')

all_scores